### **- Using analyst's eps, revenue and other data predictions to determine whether the stock price will rise or not.**

#### Here I am using analyst estimations for eps, revenue, ebitda, sga and other data, combined with the real reports on these values, to predict whether the stock price moved significantly in the following 3 months from when the above mentioned reports released.

#### For better results I am using only Hold and Buy as values for the target column, since I am taking the maximum **close** value of the stock price for the following 3 months.

In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.impute import KNNImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler

2024-09-18 23:02:19.475106: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-18 23:02:19.475705: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-18 23:02:19.477773: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-18 23:02:19.483737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-18 23:02:19.493396: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
stock_symbol = 'AAPL'
analyst_est = pd.read_json('AnalystData/' + stock_symbol + '_EST.json')
analyst_est_df = pd.DataFrame(analyst_est)
analyst_est_df['dym'] = analyst_est_df['date'].dt.to_period('M')
analyst_est_df

,symbol,date,estimatedRevenueLow,estimatedRevenueHigh,estimatedRevenueAvg,estimatedEbitdaLow,estimatedEbitdaHigh,estimatedEbitdaAvg,estimatedEbitLow,estimatedEbitHigh,...,estimatedNetIncomeAvg,estimatedSgaExpenseLow,estimatedSgaExpenseHigh,estimatedSgaExpenseAvg,estimatedEpsAvg,estimatedEpsHigh,estimatedEpsLow,numberAnalystEstimatedRevenue,numberAnalystsEstimatedEps,dym
0,AAPL,2026-09-30,101813788248,101813788248,101813788248,31840784333,31840784333,31840784333,28225198292,28225198292,...,28599968885,6748768648,6748768648,6748768648,1.84937,1.849370,1.849370,9,7,2026-09
1,AAPL,2026-06-30,93000131173,93000131173,93000131173,29084441023,29084441023,29084441023,25781843390,25781843390,...,25317275105,6164551780,6164551780,6164551780,1.63710,1.637100,1.637100,9,11,2026-06
2,AAPL,2026-03-30,102059246829,102059246829,102059246829,31917547942,31917547942,31917547942,28293245236,28293245236,...,28480735978,6765038971,6765038971,6765038971,1.84166,1.841660,1.841660,9,7,2026-03
3,AAPL,2025-12-30,127755972685,127755972685,127755972685,39953825936,39953825936,39953825936,35416987464,35416987464,...,38925136496,8468356967,8468356967,8468356967,2.51703,2.517030,2.517030,13,13,2025-12
4,AAPL,2025-09-30,99713448399,99713448399,99713448399,31183933534,31183933534,31183933534,27642934242,27642934242,...,26843951178,6609546761,6609546761,6609546761,1.73582,1.735820,1.735820,13,8,2025-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,AAPL,1996-06-29,3486400000,5229600000,4358000000,-27540000,-18360000,-22950000,-187920000,-125280000,...,-43200000,393120000,589680000,491400000,-0.01000,-0.010529,-0.020000,15,15,1996-06
105,AAPL,1996-03-30,1248571427,1872857141,1560714285,-883871999,-589247999,-736559999,-936143999,-624095999,...,-488399999,213312000,319967999,266639999,-0.01000,-0.009131,-0.010529,9,9,1996-03
106,AAPL,1995-12-30,1259200000,1888800000,1574000000,-66000000,-44000000,-55000000,-157080000,-104720000,...,-75900000,388080000,582120000,485100000,0.00000,0.000000,0.000000,12,12,1995-12
107,AAPL,1995-09-29,3783780000,5675670000,4729725000,106400000,159600000,133000000,57600000,86400000,...,60000000,302400000,453600000,378000000,0.01000,0.010529,0.009131,15,15,1995-09


In [3]:
income = pd.read_json('AnalystData/' + stock_symbol + '_INCOME_STATEMENT.json')
income_df = pd.DataFrame(income)
income_df['fillingDate'] = pd.to_datetime(income_df['fillingDate'])
income_df['dym'] = income_df['date'].dt.to_period('M')
income_df['fillingDym'] = income_df['fillingDate'].dt.to_period('M')

columns_to_keep = ['revenue', 'sellingGeneralAndAdministrativeExpenses', 'ebitda', 'epsdiluted', 'eps', 'netIncome', 'dym', 'fillingDym']

income_df = income_df[columns_to_keep]
income_df

,revenue,sellingGeneralAndAdministrativeExpenses,ebitda,epsdiluted,eps,netIncome,dym,fillingDym
0,85777000000,6320000000,28202000000,1.4000,1.4000,21448000000,2024-06,2024-08
1,90753000000,6468000000,30736000000,1.5300,1.5300,23636000000,2024-03,2024-05
2,119575000000,6786000000,43221000000,2.1800,2.1900,33916000000,2023-12,2024-02
3,89498000000,6151000000,29622000000,1.4600,1.4700,22956000000,2023-09,2023-11
4,81797000000,5973000000,26050000000,1.2600,1.2700,19881000000,2023-07,2023-08
...,...,...,...,...,...,...,...,...
151,510800000,0,-1117600000,0.0086,0.0086,0,1986-09,1986-09
152,448300000,0,448300000,0.0023,0.0023,0,1986-06,1986-06
153,408900000,0,408900000,0.0023,0.0023,0,1986-03,1986-03
154,533900000,0,533900000,0.0041,0.0041,0,1985-12,1985-12


In [4]:
est_min_date = analyst_est_df['date'].min()
today = datetime.date.today()
stock_prices_df = yf.download(stock_symbol, est_min_date, today)

stock_prices_df.drop(columns=['Open', 'High', 'Low', 'Volume', 'Close'], inplace=True)
stock_prices_df

[*********************100%%**********************]  1 of 1 completed


,Adj Close
Date,
1995-06-29,0.354190
1995-06-30,0.348100
1995-07-03,0.351847
1995-07-05,0.348568
1995-07-06,0.352316
...,...
2024-09-11,222.660004
2024-09-12,222.770004
2024-09-13,222.500000



#### - Resample by month and find the maximum stock price for each month, then find the maximum for the next 3 months by shifting and rolling
#### - Then create the dym(date in year-month format) column for merging the dataframes on the months

In [5]:
monthly_max_df = stock_prices_df['Adj Close'].resample('M').max().to_frame()
monthly_max_df['max_next_3'] = (
    monthly_max_df['Adj Close']
    .shift(-1)
    .rolling(window=3, min_periods=1)
    .max()
)

monthly_max_df['dym'] = monthly_max_df.index.to_period('M')
monthly_max_df = monthly_max_df.drop(columns='Adj Close')
monthly_max_df

,max_next_3,dym
Date,,
1995-06-30,0.367308,1995-06
1995-07-31,0.367308,1995-07
1995-08-31,0.367308,1995-08
1995-09-30,0.343882,1995-09
1995-10-31,0.336366,1995-10
...,...,...
2024-05-31,216.419495,2024-05
2024-06-30,234.548523,2024-06
2024-07-31,234.548523,2024-07


#### - Merging the analysts estimations(analyst_est_df) together with real reports(income_df)
#### - Then merging this with the prices and max prices.

In [6]:
merged_est_income_df = pd.merge(analyst_est_df, income_df, how='left', left_on='dym', right_on='dym')
merged_df = pd.merge(merged_est_income_df, monthly_max_df, how='left', left_on='fillingDym', right_on='dym')
merged_df = merged_df.sort_values('fillingDym', ascending=False)
merged_df = merged_df.dropna(subset=['revenue'])
merged_df.reset_index(inplace=True)
merged_df

,index,symbol,date,estimatedRevenueLow,estimatedRevenueHigh,estimatedRevenueAvg,estimatedEbitdaLow,estimatedEbitdaHigh,estimatedEbitdaAvg,estimatedEbitLow,...,dym_x,revenue,sellingGeneralAndAdministrativeExpenses,ebitda,epsdiluted,eps,netIncome,fillingDym,max_next_3,dym_y
0,9,AAPL,2024-06-30,82225628203,87077794168,84174531303,25714871623,27232316094,26324362777,22794895472,...,2024-06,8.577700e+10,6.320000e+09,2.820200e+10,1.4000,1.4000,2.144800e+10,2024-08,234.548523,2024-08
1,10,AAPL,2024-03-30,90366105851,90366105851,90366105851,30447851074,45671776615,38059813845,27193259076,...,2024-03,9.075300e+10,6.468000e+09,3.073600e+10,1.5300,1.5300,2.363600e+10,2024-05,216.419495,2024-05
2,11,AAPL,2023-12-30,116943355001,119691894997,117986596460,36572336713,37431902699,36898595335,32419473243,...,2023-12,1.195750e+11,6.786000e+09,4.322100e+10,2.1800,2.1900,3.391600e+10,2024-02,194.442154,2024-02
3,12,AAPL,2023-09-30,87886970178,92222984373,89343842078,27485374145,28841399643,27940989684,24364353819,...,2023-09,8.949800e+10,6.151000e+09,2.962200e+10,1.4600,1.4700,2.295600e+10,2023-11,197.361069,2023-11
4,15,AAPL,2022-12-30,121333299852,121333299852,121333299852,37945228240,37945228240,37945228240,33636470135,...,2022-12,1.171540e+11,6.607000e+09,3.893200e+10,1.8800,1.8900,2.999800e+10,2023-02,163.612946,2023-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,104,AAPL,1996-06-29,3486400000,5229600000,4358000000,-27540000,-18360000,-22950000,-187920000,...,1996-06,2.179000e+09,3.640000e+08,-1.500000e+07,-0.0023,-0.0023,-3.200000e+07,1996-08,0.187556,1996-08
94,105,AAPL,1996-03-30,1248571427,1872857141,1560714285,-883871999,-589247999,-736559999,-936143999,...,1996-03,2.185000e+09,4.040000e+08,-9.160000e+08,-0.0535,-0.0535,-7.400000e+08,1996-05,0.214888,1996-05
95,106,AAPL,1995-12-30,1259200000,1888800000,1574000000,-66000000,-44000000,-55000000,-157080000,...,1995-12,3.148000e+09,4.410000e+08,-6.100000e+07,-0.0050,-0.0050,-6.900000e+07,1996-02,0.263899,1996-02
96,107,AAPL,1995-09-29,3783780000,5675670000,4729725000,106400000,159600000,133000000,57600000,...,1995-09,3.003000e+09,3.780000e+08,1.200000e+08,0.0043,0.0043,6.000000e+07,1995-12,0.311937,1995-12


#### - In the following cell I am creating the target column **trend**. We have percentage_limit that defines what should our criteria be for determening whether the stock price has risen significantly or not. I tried different options for this value, and it seems it depends what kind of stock we have, and how much data we have. It works differently for diffent kind of scenarios. I would maybe improve this part by cross validating this value and see for what value perhaps from 0 to 10 the models give the best result.

#### - Calculating the percentage change from the next row and the current row, then creating an interval that defines our Hold or Buy target column using the pct_change column.

In [7]:
percentage_limit = 0
merged_df['pct_change'] = (merged_df['max_next_3'].shift(1) - merged_df['max_next_3'] ) / merged_df['max_next_3'].shift(1) * 100

conditions = [
    (merged_df['pct_change'] >= percentage_limit),   
    (merged_df['pct_change'] < -percentage_limit) 
    # (merged_df['pct_change'].between(-percentage_limit, percentage_limit)) 
]

# choices = ['Buy', 'Hold', 'Sell']
choices = ['Buy', 'Hold']

# merged_df['trend'] = pd.cut(merged_df['pct_change'], bins=[-float('inf'), -percentage_limit, percentage_limit,
#                                                            float('inf')], labels=['Sell', 'Hold', 'Buy'])
merged_df['trend'] = pd.cut(merged_df['pct_change'], bins=[-float('inf'), percentage_limit,
                                                           float('inf')], labels=['Hold', 'Buy'])
merged_df

,index,symbol,date,estimatedRevenueLow,estimatedRevenueHigh,estimatedRevenueAvg,estimatedEbitdaLow,estimatedEbitdaHigh,estimatedEbitdaAvg,estimatedEbitLow,...,sellingGeneralAndAdministrativeExpenses,ebitda,epsdiluted,eps,netIncome,fillingDym,max_next_3,dym_y,pct_change,trend
0,9,AAPL,2024-06-30,82225628203,87077794168,84174531303,25714871623,27232316094,26324362777,22794895472,...,6.320000e+09,2.820200e+10,1.4000,1.4000,2.144800e+10,2024-08,234.548523,2024-08,NaN,NaN
1,10,AAPL,2024-03-30,90366105851,90366105851,90366105851,30447851074,45671776615,38059813845,27193259076,...,6.468000e+09,3.073600e+10,1.5300,1.5300,2.363600e+10,2024-05,216.419495,2024-05,7.729330,Buy
2,11,AAPL,2023-12-30,116943355001,119691894997,117986596460,36572336713,37431902699,36898595335,32419473243,...,6.786000e+09,4.322100e+10,2.1800,2.1900,3.391600e+10,2024-02,194.442154,2024-02,10.154973,Buy
3,12,AAPL,2023-09-30,87886970178,92222984373,89343842078,27485374145,28841399643,27940989684,24364353819,...,6.151000e+09,2.962200e+10,1.4600,1.4700,2.295600e+10,2023-11,197.361069,2023-11,-1.501174,Hold
4,15,AAPL,2022-12-30,121333299852,121333299852,121333299852,37945228240,37945228240,37945228240,33636470135,...,6.607000e+09,3.893200e+10,1.8800,1.8900,2.999800e+10,2023-02,163.612946,2023-02,17.099686,Buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,104,AAPL,1996-06-29,3486400000,5229600000,4358000000,-27540000,-18360000,-22950000,-187920000,...,3.640000e+08,-1.500000e+07,-0.0023,-0.0023,-3.200000e+07,1996-08,0.187556,1996-08,5.238206,Buy
94,105,AAPL,1996-03-30,1248571427,1872857141,1560714285,-883871999,-589247999,-736559999,-936143999,...,4.040000e+08,-9.160000e+08,-0.0535,-0.0535,-7.400000e+08,1996-05,0.214888,1996-05,-14.572835,Hold
95,106,AAPL,1995-12-30,1259200000,1888800000,1574000000,-66000000,-44000000,-55000000,-157080000,...,4.410000e+08,-6.100000e+07,-0.0050,-0.0050,-6.900000e+07,1996-02,0.263899,1996-02,-22.807202,Hold
96,107,AAPL,1995-09-29,3783780000,5675670000,4729725000,106400000,159600000,133000000,57600000,...,3.780000e+08,1.200000e+08,0.0043,0.0043,6.000000e+07,1995-12,0.311937,1995-12,-18.203277,Hold


In [8]:
ready_df = merged_df.drop(columns=['date', 'dym_x', 'dym_y', 'fillingDym', 'symbol', 'pct_change', 'max_next_3', 'index'], inplace=False)
ready_df = ready_df.dropna(subset=['trend'])
ready_df

,estimatedRevenueLow,estimatedRevenueHigh,estimatedRevenueAvg,estimatedEbitdaLow,estimatedEbitdaHigh,estimatedEbitdaAvg,estimatedEbitLow,estimatedEbitHigh,estimatedEbitAvg,estimatedNetIncomeLow,...,estimatedEpsLow,numberAnalystEstimatedRevenue,numberAnalystsEstimatedEps,revenue,sellingGeneralAndAdministrativeExpenses,ebitda,epsdiluted,eps,netIncome,trend
1,90366105851,90366105851,90366105851,30447851074,45671776615,38059813845,27193259076,40789888614,33991573845,22685993844,...,1.505080,4,9,9.075300e+10,6.468000e+09,3.073600e+10,1.5300,1.5300,2.363600e+10,Buy
2,116943355001,119691894997,117986596460,36572336713,37431902699,36898595335,32419473243,33181433756,32708684533,31554192245,...,2.040400,9,11,1.195750e+11,6.786000e+09,4.322100e+10,2.1800,2.1900,3.391600e+10,Buy
3,87886970178,92222984373,89343842078,27485374145,28841399643,27940989684,24364353819,25566399854,24768233284,20871944503,...,1.349650,24,28,8.949800e+10,6.151000e+09,2.962200e+10,1.4600,1.4700,2.295600e+10,Hold
4,121333299852,121333299852,121333299852,37945228240,37945228240,37945228240,33636470135,33636470135,33636470135,29982204575,...,1.938750,11,21,1.171540e+11,6.607000e+09,3.893200e+10,1.8800,1.8900,2.999800e+10,Buy
5,88739279089,88739279089,88739279089,20127932000,30191898000,25159915000,17791928000,26687892000,22239910000,15372148000,...,1.267980,16,14,9.014600e+10,6.440000e+09,2.775900e+10,1.2900,1.2900,2.072100e+10,Buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,3486400000,5229600000,4358000000,-27540000,-18360000,-22950000,-187920000,-125280000,-156600000,-51840000,...,-0.020000,15,15,2.179000e+09,3.640000e+08,-1.500000e+07,-0.0023,-0.0023,-3.200000e+07,Buy
94,1248571427,1872857141,1560714285,-883871999,-589247999,-736559999,-936143999,-624095999,-780119999,-586079999,...,-0.010529,9,9,2.185000e+09,4.040000e+08,-9.160000e+08,-0.0535,-0.0535,-7.400000e+08,Hold
95,1259200000,1888800000,1574000000,-66000000,-44000000,-55000000,-157080000,-104720000,-130900000,-91080000,...,0.000000,12,12,3.148000e+09,4.410000e+08,-6.100000e+07,-0.0050,-0.0050,-6.900000e+07,Hold
96,3783780000,5675670000,4729725000,106400000,159600000,133000000,57600000,86400000,72000000,48000000,...,0.009131,15,15,3.003000e+09,3.780000e+08,1.200000e+08,0.0043,0.0043,6.000000e+07,Hold


In [9]:
label_encoder = LabelEncoder()
ready_df['trend'] = label_encoder.fit_transform(ready_df['trend'])
ready_df

,estimatedRevenueLow,estimatedRevenueHigh,estimatedRevenueAvg,estimatedEbitdaLow,estimatedEbitdaHigh,estimatedEbitdaAvg,estimatedEbitLow,estimatedEbitHigh,estimatedEbitAvg,estimatedNetIncomeLow,...,estimatedEpsLow,numberAnalystEstimatedRevenue,numberAnalystsEstimatedEps,revenue,sellingGeneralAndAdministrativeExpenses,ebitda,epsdiluted,eps,netIncome,trend
1,90366105851,90366105851,90366105851,30447851074,45671776615,38059813845,27193259076,40789888614,33991573845,22685993844,...,1.505080,4,9,9.075300e+10,6.468000e+09,3.073600e+10,1.5300,1.5300,2.363600e+10,0
2,116943355001,119691894997,117986596460,36572336713,37431902699,36898595335,32419473243,33181433756,32708684533,31554192245,...,2.040400,9,11,1.195750e+11,6.786000e+09,4.322100e+10,2.1800,2.1900,3.391600e+10,0
3,87886970178,92222984373,89343842078,27485374145,28841399643,27940989684,24364353819,25566399854,24768233284,20871944503,...,1.349650,24,28,8.949800e+10,6.151000e+09,2.962200e+10,1.4600,1.4700,2.295600e+10,1
4,121333299852,121333299852,121333299852,37945228240,37945228240,37945228240,33636470135,33636470135,33636470135,29982204575,...,1.938750,11,21,1.171540e+11,6.607000e+09,3.893200e+10,1.8800,1.8900,2.999800e+10,0
5,88739279089,88739279089,88739279089,20127932000,30191898000,25159915000,17791928000,26687892000,22239910000,15372148000,...,1.267980,16,14,9.014600e+10,6.440000e+09,2.775900e+10,1.2900,1.2900,2.072100e+10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,3486400000,5229600000,4358000000,-27540000,-18360000,-22950000,-187920000,-125280000,-156600000,-51840000,...,-0.020000,15,15,2.179000e+09,3.640000e+08,-1.500000e+07,-0.0023,-0.0023,-3.200000e+07,0
94,1248571427,1872857141,1560714285,-883871999,-589247999,-736559999,-936143999,-624095999,-780119999,-586079999,...,-0.010529,9,9,2.185000e+09,4.040000e+08,-9.160000e+08,-0.0535,-0.0535,-7.400000e+08,1
95,1259200000,1888800000,1574000000,-66000000,-44000000,-55000000,-157080000,-104720000,-130900000,-91080000,...,0.000000,12,12,3.148000e+09,4.410000e+08,-6.100000e+07,-0.0050,-0.0050,-6.900000e+07,1
96,3783780000,5675670000,4729725000,106400000,159600000,133000000,57600000,86400000,72000000,48000000,...,0.009131,15,15,3.003000e+09,3.780000e+08,1.200000e+08,0.0043,0.0043,6.000000e+07,1


In [10]:
X = ready_df.drop(columns='trend', inplace=False)
y = ready_df['trend']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [11]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### I tried using multiple models to see which will perform the best.

For the new data entry point, XBGClassifier gives a **Hold**, while the neural network build with Sequential gives a **Buy**, as well as the RandomForestClassifier (but this model is not performing well).

In [12]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', enable_categorical=True)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.85      0.79        13
           1       0.60      0.43      0.50         7

    accuracy                           0.70        20
   macro avg       0.67      0.64      0.64        20
weighted avg       0.69      0.70      0.69        20

Accuracy: 0.7


In [20]:
label_encoder.inverse_transform(y_pred)

array(['Buy', 'Hold', 'Buy', 'Buy', 'Hold', 'Buy', 'Buy', 'Hold', 'Buy',
       'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy',
       'Hold', 'Hold'], dtype=object)

#### Here I am predicting the trend for the latest informations for which we don't have the future price information.

In [14]:
new_data = merged_df.head(1).drop(columns=['date', 'dym_x', 'dym_y', 'fillingDym', 'symbol', 'pct_change', 'max_next_3', 'index', 'trend'], inplace=False)

predicted_trend_encoded = model.predict(new_data)
predicted_trend = label_encoder.inverse_transform(predicted_trend_encoded)

print("Predicted Trend:", predicted_trend[0])

Predicted Trend: Hold


In [15]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')  # 2 classes: Buy, Hold
    # Dense(3, activation='softmax')  # 3 classes: Buy, Hold, Sell
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2)


Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5187 - loss: 0.6937 - val_accuracy: 0.5625 - val_loss: 0.6832
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6579 - loss: 0.6573 - val_accuracy: 0.6250 - val_loss: 0.6791
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6746 - loss: 0.6373 - val_accuracy: 0.6250 - val_loss: 0.6763
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7273 - loss: 0.6325 - val_accuracy: 0.6250 - val_loss: 0.6753
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6530 - loss: 0.6610 - val_accuracy: 0.6250 - val_loss: 0.6762
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7080 - loss: 0.5913 - val_accuracy: 0.6250 - val_loss: 0.6785
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6415 - loss: 0.7164 - val_accuracy: 0.6250 - val_loss: 0.6785
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7080 - loss: 0.6162 - val_accuracy: 0.6250 - val_loss: 0.6797
Epoch 9

In [16]:
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# print(classification_report(y_test, y_pred, target_names=['Buy', 'Hold', 'Hold']))
print(classification_report(y_test, y_pred, target_names=['Buy', 'Hold']))
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
              precision    recall  f1-score   support

         Buy       0.65      1.00      0.79        13
        Hold       0.00      0.00      0.00         7

    accuracy                           0.65        20
   macro avg       0.33      0.50      0.39        20
weighted avg       0.42      0.65      0.51        20

[[13  0]
 [ 7  0]]
Accuracy: 0.65


/home/meto/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/meto/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/meto/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
predicted_trend_encoded = model.predict(new_data)
y_pred_new_data = np.argmax(predicted_trend_encoded, axis=1)
predicted_trend = label_encoder.inverse_transform(y_pred_new_data)

print("Predicted Trend:", predicted_trend[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Trend: Buy


In [18]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report

smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

model = RandomForestClassifier(n_estimators=100, max_depth=43, random_state=42)
model.fit(X_train_resampled, y_train_resampled)

y_pred = model.predict(X_test)

# print(classification_report(y_test, y_pred, target_names=['Buy', 'Hold', 'Hold']))
print(classification_report(y_test, y_pred, target_names=['Buy', 'Hold']))
print("Accuracy:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         Buy       0.60      0.69      0.64        13
        Hold       0.20      0.14      0.17         7

    accuracy                           0.50        20
   macro avg       0.40      0.42      0.40        20
weighted avg       0.46      0.50      0.48        20

Accuracy: 0.5


In [19]:
predicted_trend_encoded = model.predict(new_data)
predicted_trend = label_encoder.inverse_transform(predicted_trend_encoded)

print("Predicted Trend:", predicted_trend[0])

Predicted Trend: Buy


/home/meto/.local/lib/python3.11/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
